In [1]:
import pandas as pd
import pickle

In [93]:
!pip install html2text

In [2]:
import html2text

In [2]:
pd.set_option('display.max_colwidth', 5000)

In [4]:
df = pd.read_pickle('/Users/hazem/Documents/manzur/data/pickles/enabbaladi/posts/posts_1_0.pkl')

In [5]:
df.columns

Index(['_links', 'amp_enabled', 'amp_validity', 'author', 'categories',
       'comment_status', 'content', 'date', 'date_gmt', 'excerpt',
       'featured_media', 'format', 'guid', 'id', 'jetpack-related-posts',
       'jetpack_featured_media_url', 'jetpack_shortlink', 'link', 'meta',
       'modified', 'modified_gmt', 'ping_status', 'slug', 'status', 'sticky',
       'tags', 'template', 'title', 'type', 'yoast_head'],
      dtype='object')

In [62]:
def read_tags(tag_file_num=9):
    tags = dict()
    for i in range(tag_file_num):
        file = f'/Users/hazem/Documents/manzur/data/pickles/enabbaladi/tags/tag_{i}.pkl'
        new_tags = read_tags_file(file)
        tags = {**tags, **new_tags}
    return tags

tags = read_tags(8)
len(tags)

38913

In [61]:
def read_tags_file(file):
    cats = dict()
    df = pd.read_pickle(file)
    df['id'] = df['id'].apply(int)
    df['id'] = df['id'].apply(str)
    df = df[['id','name', 'count']]
    df = df.set_index('id')
    names = df.to_dict()['name']
    counts = df.to_dict()['count']
    for k in names:
        if counts.get(k,0) >=1:
            cats[k.strip()] = {'name': names[k], 'count': int(counts.get(k,0))}
    return cats

file = f'/Users/hazem/Documents/manzur/data/pickles/enabbaladi/categories.pkl'
cats = read_tags_file(file)
len(cats)

80

In [135]:
def clean_post_file(file):
    h = html2text.HTML2Text()
    h.ignore_links = True
    df = pd.read_pickle(file)
    df['title'] = df['title'].apply(lambda x: dict(x)['rendered'].strip())
    df['title'] = df['title'].apply(h.handle)
    df['content'] = df['content'].apply(lambda x: dict(x)['rendered'])
    df['html_content'] = df['content']
    df['content'] = df['content'].apply(h.handle)
    df['content'] = df['content'].str.replace('\n', ' ')
    df['excerpt'] = df['excerpt'].apply(lambda x: dict(x)['rendered'])
    df['excerpt'] = df['excerpt'].apply(h.handle)
    df['tags_str'] = df['tags'].apply(lambda x: '|'.join([tags.get(str(e), dict()).get('name', str(e)) for e in list(x)]))
    df['categories_str'] = df['categories'].apply(lambda x: '|'.join([cats.get(str(e), dict()).get('name', str(e)) for e in list(x)]))
    df['author'] = df['author'].apply(int).apply(str) 
    df['id'] = df['id'].apply(int).apply(str) 
    df['realted_posts'] = df['jetpack-related-posts'].apply(lambda x: '|'.join([str(e['id']) for e in list(x)]))
    
    return df[['id','title','content', 'html_content', 'excerpt','categories','author','date_gmt', 
                'tags','tags_str', 'categories_str', 'realted_posts']]


In [136]:
file = '/Users/hazem/Documents/manzur/data/pickles/enabbaladi/posts/posts_1_0.pkl'
df = clean_post_file(file)

In [140]:
! pip install requests

     |████████████████████████████████| 61 kB 2.9 MB/s eta 0:00:011
     |████████████████████████████████| 58 kB 4.4 MB/s eta 0:00:011
     |████████████████████████████████| 137 kB 4.7 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 178 kB 5.4 MB/s eta 0:00:01


In [9]:
import json
import requests
url = 'https://farasa.qcri.org/webapi/segmentation/'
text = 'يُشار إلى أن اللغة العربية' 
api_key = 'sHjnqGXGGNdyLianPJ'
payload = {'text': text, 'api_key': api_key}
headers = {'content-type': 'application/json'} 
data = requests.post(url, data=json.dumps(payload), headers=headers)
result = data.text
print(result) 

<!DOCTYPE html>
<html lang="en">
<head>
  <meta http-equiv="content-type" content="text/html; charset=utf-8">
  <meta name="robots" content="NONE,NOARCHIVE">
  <title>DoesNotExist
          at /webapi/segmentation/</title>
  <style type="text/css">
    html * { padding:0; margin:0; }
    body * { padding:10px 20px; }
    body * * { padding:0; }
    body { font:small sans-serif; background-color:#fff; color:#000; }
    body>div { border-bottom:1px solid #ddd; }
    h1 { font-weight:normal; }
    h2 { margin-bottom:.8em; }
    h2 span { font-size:80%; color:#666; font-weight:normal; }
    h3 { margin:1em 0 .5em 0; }
    h4 { margin:0 0 .5em 0; font-weight: normal; }
    code, pre { font-size: 100%; white-space: pre-wrap; }
    table { border:1px solid #ccc; border-collapse: collapse; width:100%; background:white; }
    tbody td, tbody th { vertical-align:top; padding:2px 3px; }
    thead th {
      padding:1px 6px 1px 3px; background:#fefefe; text-align:left;
      font-weight:normal; fo

In [10]:
! pip install farasapy

     |████████████████████████████████| 72 kB 1.9 MB/s eta 0:00:01


In [11]:
from farasa.pos import FarasaPOSTagger 
from farasa.ner import FarasaNamedEntityRecognizer 
from farasa.diacratizer import FarasaDiacritizer 
from farasa.segmenter import FarasaSegmenter 
from farasa.stemmer import FarasaStemmer

In [16]:
stemmer = FarasaStemmer()

sample = ''' 
يُشار إلى أن اللغة العربية يتحدثها أكثر من 422 مليون نسمة ويتوزع متحدثوها
 في المنطقة المعروفة باسم الوطن العربي بالإضافة إلى العديد من المناطق ال
أخرى المجاورة مثل الأهواز وتركيا وتشاد والسنغال وإريتريا وغيرها.وهي اللغ
ة الرابعة من لغات منظمة الأمم المتحدة الرسمية الست. 
'''
stemmed_text = stemmer.stem(sample)                                     
print(stemmed_text)


أشار إلى أن لغة عربي تحدث أكثر من 422 مليون نسمة توزع متحدثوها
في منطقة معروف اسم وطن عربي إضافة إلى عديد من منطقة ال
آخر مجاور مثل أهواز تركيا تشاد سنغال أريتريا غير . هي لغ
رابع من لغة منظمة أمة متحد رسمي ست .


In [17]:
postagger = FarasaPOSTagger()

In [25]:
postagged_text = postagger.tag(sample)                                     

In [26]:
postagged_text

'S/S يشار/V إلى/PREP أن/PART ال+ لغ +ة/DET+NOUN+NSUFF-FS ال+ عربي +ة/DET+ADJ+NSUFF-FS يتحدث/V +ها/PRON أكثر/ADJ-MS من/PREP 422/NUM-MP مليون/NUM-MS نسم +ة/NOUN+NSUFF-FS و+/CONJ يتوزع/V متحدثوها/NOUN-MS E/E \nS/S في/PREP ال+ منطق +ة/DET+NOUN+NSUFF-FS ال+ معروف +ة/DET+ADJ+NSUFF-FP باسم/NOUN-MS ال+ وطن/DET+NOUN-MS ال+ عربي/DET+ADJ-MS ب+/PREP ال+ إضاف +ة/DET+NOUN+NSUFF-FS إلى/PREP ال+ عديد/DET+NOUN-MS من/PREP ال+ مناطق/DET+NOUN-MS ال/DET E/E \nS/S أخرى/NOUN-MS ال+ مجاور +ة/DET+ADJ+NSUFF-FD مثل/NOUN-MS ال+ أهواز/DET+NOUN-MS و+/CONJ تركيا/NOUN-MS و+/CONJ تشاد/NOUN-MS و+/CONJ ال+ سنغال/DET+NOUN-MS و+/CONJ إريتريا/NOUN-MS و+/CONJ غير/NOUN-MS +ها/PRON ./PUNC و+/CONJ هي/PRON ال+ لغ/DET+NOUN-MS E/E \nS/S ة/NOUN-MS ال+ رابع +ة/DET+ADJ+NSUFF-FP من/PREP لغ +ات/NOUN+NSUFF-FP منظم +ة/NOUN+NSUFF-FS ال+ أمم/DET+NOUN-MS ال+ متحد +ة/DET+ADJ+NSUFF-FP ال+ رسمي +ة/DET+ADJ+NSUFF-FD ال+ ست/DET+NOUN-MS ./PUNC E/E'

In [33]:
for txt in postagged_text.split('\n'):
    row_pos_tags = txt.split('/')
    pos_tags = list()
    for i, e in enumerate(row_pos_tags):
        
        if i == 0:
            token = e
        else:
            pos_tag = e.split()[0]
            tokens, poss = token.split('+'), pos_tag.split('+')
            tokens = [t for t in tokens if t]
            poss = [p for p in poss if p]
            if len(tokens) == len(poss):
                for ii in range(len(tokens)):
                    pos_tags.append((tokens[ii], poss[ii]))
            else:
                print('error:',tokens, poss)
            token = ' '.join(e.split()[1:])
    print(pos_tags)

[('S', 'S'), ('يشار', 'V'), ('إلى', 'PREP'), ('أن', 'PART'), ('ال', 'DET'), (' لغ ', 'NOUN'), ('ة', 'NSUFF-FS'), ('ال', 'DET'), (' عربي ', 'ADJ'), ('ة', 'NSUFF-FS'), ('يتحدث', 'V'), ('ها', 'PRON'), ('أكثر', 'ADJ-MS'), ('من', 'PREP'), ('422', 'NUM-MP'), ('مليون', 'NUM-MS'), ('نسم ', 'NOUN'), ('ة', 'NSUFF-FS'), ('و', 'CONJ'), ('يتوزع', 'V'), ('متحدثوها', 'NOUN-MS'), ('E', 'E')]
[('S', 'S'), ('في', 'PREP'), ('ال', 'DET'), (' منطق ', 'NOUN'), ('ة', 'NSUFF-FS'), ('ال', 'DET'), (' معروف ', 'ADJ'), ('ة', 'NSUFF-FP'), ('باسم', 'NOUN-MS'), ('ال', 'DET'), (' وطن', 'NOUN-MS'), ('ال', 'DET'), (' عربي', 'ADJ-MS'), ('ب', 'PREP'), ('ال', 'DET'), (' إضاف ', 'NOUN'), ('ة', 'NSUFF-FS'), ('إلى', 'PREP'), ('ال', 'DET'), (' عديد', 'NOUN-MS'), ('من', 'PREP'), ('ال', 'DET'), (' مناطق', 'NOUN-MS'), ('ال', 'DET'), ('E', 'E')]
[('S', 'S'), ('أخرى', 'NOUN-MS'), ('ال', 'DET'), (' مجاور ', 'ADJ'), ('ة', 'NSUFF-FD'), ('مثل', 'NOUN-MS'), ('ال', 'DET'), (' أهواز', 'NOUN-MS'), ('و', 'CONJ'), ('تركيا', 'NOUN-MS'), ('و'

In [34]:
namedEntityRecognizer = FarasaNamedEntityRecognizer()
namedEntityRecognizer.recognize(sample)

'يشار/O إلى/O أن/O اللغة/O العربية/O يتحدثها/O أكثر/O من/O 422/O مليون/O نسمة/O ويتوزع/O متحدثوها/O\nفي/O المنطقة/O المعروفة/O باسم/O الوطن/B-ORG العربي/I-ORG بالإضافة/O إلى/O العديد/O من/O المناطق/O ال/O\nأخرى/O المجاورة/O مثل/O الأهواز/B-LOC وتركيا/B-LOC وتشاد/B-LOC والسنغال/B-LOC وإريتريا/B-LOC وغيرها/O ./O وهي/O اللغ/O\nة/O الرابعة/O من/O لغات/O منظمة/B-ORG الأمم/I-ORG المتحدة/I-ORG الرسمية/O الست/O ./O'

In [35]:
farasaStemmer = FarasaStemmer()
farasaStemmer.stem(sample)
                

'أشار إلى أن لغة عربي تحدث أكثر من 422 مليون نسمة توزع متحدثوها\nفي منطقة معروف اسم وطن عربي إضافة إلى عديد من منطقة ال\nآخر مجاور مثل أهواز تركيا تشاد سنغال أريتريا غير . هي لغ\nرابع من لغة منظمة أمة متحد رسمي ست .'